In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time
import os

In [ ]:
def get_stock_screenshots(symbol, timeframes=['1d', '5d', '1mo', '6mo', '1y', '5y']):
    # Setup Chrome in headless mode
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1920,1080")
    
    driver = webdriver.Chrome(options=chrome_options)
    
    # Create directory for screenshots
    if not os.path.exists('stock_charts'):
        os.makedirs('stock_charts')
    
    try:
        # Load Yahoo Finance page for the symbol
        url = f"https://finance.yahoo.com/quote/{symbol}/chart"
        driver.get(url)
        
        # Wait for chart to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "chart-container"))
        )
        
        # Get screenshots for each timeframe
        for timeframe in timeframes:
            # Click timeframe button
            timeframe_btn = driver.find_element(By.CSS_SELECTOR, f"[data-range='{timeframe}']")
            timeframe_btn.click()
            
            # Wait for chart to update
            time.sleep(2)
            
            # Take screenshot
            chart = driver.find_element(By.ID, "chart-container")
            screenshot_path = f"stock_charts/{symbol}_{timeframe}.png"
            chart.screenshot(screenshot_path)
            print(f"Saved {screenshot_path}")
            
    finally:
        driver.quit()

# Example usage
get_stock_screenshots('AAPL')

In [ ]:
def get_stock_screenshots(symbol, timeframes=['1d', '5d', '1mo', '6mo', '1y', '5y'], max_retries=3):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--no-sandbox")
    
    driver = webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, 20)  # Increased timeout to 20 seconds
    
    if not os.path.exists('stock_charts'):
        os.makedirs('stock_charts')
    
    try:
        url = f"https://finance.yahoo.com/quote/{symbol}/chart"
        driver.get(url)
        
        # Handle cookie consent if present
        try:
            cookie_button = wait.until(EC.presence_of_element_located((By.NAME, "agree")))
            cookie_button.click()
        except TimeoutException:
            pass  # No cookie banner found
        
        # Wait for chart container
        chart_container = wait.until(
            EC.presence_of_element_located((By.ID, "chart-container"))
        )
        
        for timeframe in timeframes:
            retries = 0
            while retries < max_retries:
                try:
                    # Wait for and click timeframe button
                    timeframe_btn = wait.until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, f"[data-range='{timeframe}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", timeframe_btn)
                    driver.execute_script("arguments[0].click();", timeframe_btn)
                    
                    # Wait for chart to update
                    time.sleep(3)
                    
                    # Take screenshot
                    chart = wait.until(EC.presence_of_element_located((By.ID, "chart-container")))
                    screenshot_path = f"stock_charts/{symbol}_{timeframe}.png"
                    chart.screenshot(screenshot_path)
                    print(f"Saved {screenshot_path}")
                    break
                    
                except (TimeoutException, ElementClickInterceptedException) as e:
                    retries += 1
                    print(f"Retry {retries} for {timeframe} due to {str(e)}")
                    if retries == max_retries:
                        print(f"Failed to capture {timeframe} after {max_retries} attempts")
                    time.sleep(2)
    
    finally:
        driver.quit()

# Example usage
get_stock_screenshots('AAPL')

In [3]:
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr

# Market Data 
import yfinance as yf

#Graphing/Visualization
import datetime as dt 
import plotly.graph_objs as go 

# Override Yahoo Finance 
# yf.pdr_override()

# Create input field for our desired stock 
stock="AAPL"

# Retrieve stock data frame (df) from yfinance API at an interval of 1m 
df = yf.download(tickers=stock,period='5d',interval='1m')

print(df)

# Declare plotly figure (go)
fig=go.Figure()

fig.add_trace(go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'], name = 'market data'))

fig.update_layout(
    title= str(stock)+' Live Share Price:',
    yaxis_title='Stock Price (USD per Shares)')               

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=15, label="15m", step="minute", stepmode="backward"),
            dict(count=45, label="45m", step="minute", stepmode="backward"),
            dict(count=1, label="HTD", step="hour", stepmode="todate"),
            dict(count=3, label="3h", step="hour", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.show()

[*********************100%***********************]  1 of 1 completed

Price                       Adj Close       Close        High         Low  \
Ticker                           AAPL        AAPL        AAPL        AAPL   
Datetime                                                                    
2024-11-04 14:30:00+00:00  222.009995  222.009995  222.399994  221.011993   
2024-11-04 14:31:00+00:00  221.529999  221.529999  222.330002  221.074493   
2024-11-04 14:32:00+00:00  220.830093  220.830093  221.690002  220.800003   
2024-11-04 14:33:00+00:00  221.360001  221.360001  221.410004  220.380005   
2024-11-04 14:34:00+00:00  220.664993  220.664993  221.339996  220.559998   
...                               ...         ...         ...         ...   
2024-11-08 20:55:00+00:00  227.024994  227.024994  227.149994  226.850006   
2024-11-08 20:56:00+00:00  227.020004  227.020004  227.130005  227.000000   
2024-11-08 20:57:00+00:00  227.009995  227.009995  227.050003  226.910004   
2024-11-08 20:58:00+00:00  227.085007  227.085007  227.119995  226.967697   